In [1]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed package Microsoft.DotNet.Interactive.ExtensionLab version 1.0.0-beta.20630.2

Loaded Microsoft.DotNet.Interactive.ExtensionLab.DataFrameKernelExtension

Added the `#!linqify` magic command.

Loaded Microsoft.DotNet.Interactive.ExtensionLab.InspectExtension

Loaded Microsoft.DotNet.Interactive.ExtensionLab.NteractKernelExtension

Added the `Explore` extension method, which you can use with `IEnumerable<T>` and `IDataView` to view data using the [nteract Data Explorer](https://github.com/nteract/data-explorer).

Loaded Microsoft.DotNet.Interactive.ExtensionLab.RecordTranscriptExtension

Loaded Microsoft.DotNet.Interactive.ExtensionLab.SqlKernelsExtension


Added `mssql` and `sqlite` to the connection types available using the [`#!connect`](https://github.com/dotnet/interactive/blob/main/docs/connect.md) magic command.

In [1]:
#r "nuget:Microsoft.Data.Sqlite,3.1.7"

Installed package Microsoft.Data.Sqlite version 3.1.7

In [1]:
using Microsoft.DotNet.Interactive.ExtensionLab;
DataExplorerExtensions.Settings.UseUri(@"https://colombod.github.io/dotnet-interactive-cdn/extensionlab/1.0.0/nteract/index.js");

In [1]:
 var data = new[]
            {
                new {Type="orange", Price=1.2},
                new {Type="apple" , Price=1.3},
                new {Type="grape" , Price=1.4}
            };

In [1]:
data.Explore()

# .NET Interactive ExtensionLab: SQL Connections

This sample demonstrates how to use the `#!connect` extension

In [1]:
#!connect sqlite -h

sqlite:
  Connects to a SQLite database

Usage:
  #!connect sqlite [options] <connectionString>

Arguments:
  <connectionString>    The connection string used to connect to the database

Options:
  --kernel-name <kernel-name>    The name of the subkernel to be added
  -?, -h, --help                 Show help and usage information



In [1]:
#!connect sqlite --kernel-name collections "Data Source=C:\Temp\collectionsSQLite;"

Kernel added: #!collections

In [1]:
#!collections  -h


#!collections:
  Run the code that follows using the collections kernel. (Connected kernel)

Usage:
  #!collections [options]

Options:
  -?, -h, --help    Show help and usage information



In [1]:
#!collections 
SELECT name, type FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1;

In [1]:
#!collections
 SELECT   count(parent_id), 
          collections.title as CollectionTitle 
 FROM                    collections_items_relationship 
 JOIN     items       ON collections_items_relationship.item_id = items.id  
 JOIN     collections ON collections_items_relationship.parent_id = collections.id
 GROUP BY collections.id

# .NET Interactive ExtensionLab: Microsoft.Data.Analysis

This section demonstrates some of the experiments in the *ExtensionLab*  relating to the `DataFrame` class from [`Microsoft.Data.Analysis`](https://www.nuget.org/packages/Microsoft.Data.Analysis/).

## The `#!linqify` magic command

The `#!linqify` magic command builds a strongly-typed wrapper class around a `Microsoft.Data.Analysis.DataFrame` instance, which lets you write LINQ code against your data.  (You can learn more about `DataFrame` [here](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/).)

To start, we'll add the `Microsoft.Data.Analysis` NuGet package.

In [1]:
#r "nuget:Microsoft.Data.Analysis,0.4.0"

In [1]:
using Microsoft.Data.Analysis;

var MyDataFrame = DataFrame.LoadCsv(@"wins.csv");

MyDataFrame.Columns

index,type,value
0,Microsoft.Data.Analysis.StringDataFrameColumn,"[ Lewis Hamilton, Michael Schumacher, Sebastian Vettel, Alain Prost, Ayrton Senna, Fernando Alonso, Nigel Mansell, Jackie Stewart, Niki Lauda, Jim Clark, Nico Rosberg, Nelson Piquet, Juan Manuel Fangio ]"
1,Microsoft.Data.Analysis.SingleDataFrameColumn,"[ 94, 92, 53, 51, 41, 32, 31, 27, 25, 25, 23, 23, 23 ]"


After running the previous cell, you can see that the `DataFrame` has columns with a few different data types. But since these are only known once the data is loaded, accessing them in a strongly-typed way isn't normally possible.

The `DataFrameRow` indexer returns `object`. So 
```c#
MyDataFrame.Rows[0][1].GetType()
```
returns `System.single`
But 

```c#
DataFrameRow row = myDataFrame.Rows[0];
Single value = row[0];
```
won't compile because the row indexer returns System.Object

This is where the `#!linqify` magic command we've installed from the ExtensionLab becomes useful. Since we know the column types in the `DataFrame` once it's been loaded, we can create a custom class with this understanding. And with .NET Interactive, we can do this at runtime, compile it, and replace the existing `MyDataFrame` variable with an instance of the new, more specific class.

`#!linqify --show-code True ` will let you see the code being used


In [1]:
#!linqify MyDataFrame

Now, you can write code to traverse the `DataFrame` using LINQ: 

In [1]:
MyDataFrame
   .OrderBy(row => row.HowMany)
   .ThenBy(row => row.WinningDriver)

index,WinningDriver,HowMany
0,Juan Manuel Fangio,23
1,Nelson Piquet,23
2,Nico Rosberg,23
3,Jim Clark,25
4,Niki Lauda,25
5,Jackie Stewart,27
6,Nigel Mansell,31
7,Fernando Alonso,32
8,Ayrton Senna,41
9,Alain Prost,51


## Visualizing the data with the nteract Data Explorer

The [nteract Data Explorer](https://blog.nteract.io/designing-the-nteract-data-explorer-f4476d53f897) is a powerful tool for understanding a dataset. Another experimental extension that we loaded when we installed the ExtensionLab package brings support for visualizing data from a number of types, including `IDataView`, which the `DataFrame` implements. The extension method `Explore` will render your data using the nteract Data Explorer:

In [1]:
using Microsoft.ML;

MyDataFrame.Explore();